In [3]:
# import numpy as np

# dataset = np.load("//Users//jayanagaprakashthota//Downloads//nyc_taxi_data.npy", allow_pickle=True).item()
# X_train, y_train, X_test, y_test = dataset["X_train"], dataset["y_train"], dataset["X_test"], dataset["y_test"]

# print("Training data shape:")
# print("X_train:", X_train.shape)
# print("y_train:", y_train.shape)
# print("Test data shape:")
# print("X_test:", X_test.shape)
# print("y_test:", y_test.shape)

In [5]:
import numpy as np

class Layer:
    def forward(self, x):
        raise NotImplementedError
    
    def backward(self, grad):
        raise NotImplementedError

class Linear(Layer):
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        self.weights = np.random.randn(output_size, input_size)
        self.bias = np.random.randn(output_size)
        self.input = None
    
    def forward(self, x):
        self.input = x
        return np.dot(x, self.weights.T) + self.bias
    
    def backward(self, grad):
        grad_input = np.dot(grad, self.weights)
        grad_weights = np.dot(grad.T, self.input)
        grad_bias = np.sum(grad, axis=0)
        return grad_input, grad_weights, grad_bias

class Sigmoid(Layer):
    def __init__(self):
        self.output = None
    
    def forward(self, x):
        self.output = 1 / (1 + np.exp(-x))
        return self.output
    
    def backward(self, grad):
        grad_input = grad * self.output * (1 - self.output)
        return grad_input

class ReLU(Layer):
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        return np.maximum(0, x)
    
    def backward(self, grad):
        grad_input = grad.copy()
        grad_input[self.mask] = 0
        return grad_input

class Sequential(Layer):
    def __init__(self):
        self.layers = []
    
    def add(self, layer):
        self.layers.append(layer)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self, grad):
        for layer in reversed(self.layers):
            grad = layer.backward(grad)
        return grad


In [6]:
# Load and preprocess the data
dataset = np.load("//Users//jayanagaprakashthota//Downloads//nyc_taxi_data.npy", allow_pickle=True).item()
X_train, y_train, X_test, y_test = dataset["X_train"], dataset["y_train"], dataset["X_test"], dataset["y_test"]

In [7]:
def train_model(model, X_train, y_train, X_val, y_val, learning_rate=0.001, epochs=100, batch_size=128):
    train_loss_history = []
    val_loss_history = []
    for epoch in range(epochs):
        # Shuffle training data
        indices = np.arange(len(X_train))
        np.random.shuffle(indices)
        X_train_shuffled = X_train[indices]
        y_train_shuffled = y_train[indices]
        
        # Mini-batch training
        for i in range(0, len(X_train_shuffled), batch_size):
            X_batch = X_train_shuffled[i:i+batch_size]
            y_batch = y_train_shuffled[i:i+batch_size]
            
            # Forward pass
            output = model.forward(X_batch)
            
            # Compute loss
            loss = np.mean((output - y_batch)**2)
            train_loss_history.append(loss)
            
            # Backward pass
            grad = 2 * (output - y_batch) / len(X_batch)
            model.backward(grad)
            
            # Update weights
            for layer in model.layers:
                if isinstance(layer, Linear):
                    layer.weights -= learning_rate * layer.grad_weights
                    layer.bias -= learning_rate * layer.grad_bias
        
        # Validate
        val_output = model.forward(X_val)
        val_loss = np.mean((val_output - y_val)**2)
        val_loss_history.append(val_loss)
        
        # Early stopping
        if epoch > 2 and val_loss_history[-1] >= val_loss_history[-2] >= val_loss_history[-3]:
            break
        
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {loss:.4f}, Val Loss: {val_loss:.4f}")
    
    return train_loss_history, val_loss_history


In [8]:



# Define and train the neural networks

# Model 1: Linear -> Sigmoid -> Linear
model1 = Sequential()
model1.add(Linear(input_size=10, output_size=64))
model1.add(Sigmoid())
model1.add(Linear(input_size=64, output_size=1))

train_loss_history1, val_loss_history1 = train_model(model1, X_train, y_train, X_test, y_test)



KeyError: "None of [Index([ 630068,  907068, 1120451,  813547,  317666,  669205, 1028569, 1063512,\n        620664,  664395,\n       ...\n       1001046,   48149,  255485, 1238917,  394567,  479028,  941570,  272089,\n       1140720,  345300],\n      dtype='int64', length=1312779)] are in the [columns]"

In [ ]:
# Model 2: Linear -> ReLU -> Linear
model2 = Sequential()
model2.add(Linear(input_size=10, output_size=64))
model2.add(ReLU())
model2.add(Linear(input_size=64, output_size=1))

train_loss_history2, val_loss_history2 = train_model(model2, X_train, y_train, X_test, y_test)



In [ ]:
# Model 3: Linear -> Sigmoid -> ReLU -> Linear
model3 = Sequential()
model3.add(Linear(input_size=10, output_size=64))
model3.add(Sigmoid())
model3.add(ReLU())
model3.add(Linear(input_size=64, output_size=1))

train_loss_history3, val_loss_history3 = train_model(model3, X_train, y_train, X_test, y_test)
